In [1]:
import pyesapi
import atexit
import SimpleITK as sitk
import os
import numpy as np

In [2]:
def get_direction(varian_image):
    x_direction = [varian_image.get_XDirection()[i] for i in range(3)]
    y_direction = [varian_image.get_YDirection()[i] for i in range(3)]
    z_direction = [varian_image.get_ZDirection()[i] for i in range(3)]
    direction = x_direction + y_direction + z_direction
    return tuple(direction)

def get_spacing(varian_image):
    return (varian_image.get_XRes(), varian_image.get_YRes(), varian_image.get_ZRes())

def get_origin(potential_image):
    return tuple([potential_image.get_Origin()[i] for i in range(3)])
def create_sitk_Image(varian_image):
    intercept = varian_image.VoxelToDisplayValue(0)
    image_array = varian_image.np_array_like().transpose()
    image_handle = sitk.GetImageFromArray(image_array + intercept)
    image_handle.SetSpacing(get_spacing(varian_image))
    image_handle.SetDirection(get_direction(varian_image))
    image_handle.SetOrigin(get_origin(varian_image))
    return image_handle
def create_sitk_Mask(varian_mask, varian_image):
    image_handle = sitk.GetImageFromArray(varian_mask.np_mask_like(varian_image).transpose())
    image_handle.SetSpacing(get_spacing(varian_image))
    image_handle.SetDirection(get_direction(varian_image))
    image_handle.SetOrigin(get_origin(varian_image))
    return image_handle

In [3]:
app = pyesapi.CustomScriptExecutable.CreateApplication('python_demo')
atexit.register(app.Dispose)

<bound method 'Dispose'>

In [4]:
fid = open(os.path.join('.', 'PatientID'))
line = fid.readline()
line = fid.readline() # Go to the second patient
fid.close()
MRN = line.strip('\n')

In [5]:
patient = app.OpenPatientById(MRN)

In [6]:
courses = [c.Id for c in patient.Courses]

In [7]:
courses

['0 QA', '1 Abdomen', 'DONOTUSE']

In [8]:
course = patient.CoursesLot(courses[-1])

In [9]:
course.Id

'DONOTUSE'

In [10]:
brachy_plans = [p for p in course.BrachyPlanSetups]

In [11]:
brachy_plans[0].Id

'DontUse1'

In [12]:
plan = brachy_plans[0]

In [14]:
plan.Id

'DontUse1'

In [16]:
catheters = [i for i in plan.Catheters]

In [17]:
catheter = catheters[0]

In [23]:
def is_a_ring(catheter):
    if catheter.ApplicatorLength != 1320.0:
        return False
    if catheter.DeadSpaceLength.is_integer():
        return False
    """
    Check to see if the z distance is greater than 1 cm
    """
    z_distances = [i.z for i in catheter.Shape] # See what the distances are between the points
    max_z_distance = np.max(z_distances) - np.min(z_distances) # If the max z distance is 5 mm, this is probably a ring
    if max_z_distance > 10:
        return False
    """
    Now check to see if we have a set of dwell times, followed by no dwell times, and then more
    """
    return is_ring_by_angle(catheter)

In [64]:
def is_ring_by_angle(catheter):
    points = np.asarray([(i.x, i.y, i.z) for i in catheter.Shape])
    angle_total = 0
    vectors = []
    for i in range(points.shape[0]-1):
        point_1 = points[i]
        point_2 = points[i+1]
        vector_1 = point_2 - point_1
        vectors.append(vector_1)
    for i in range(len(vectors) - 1):
        vector_1 = vectors[i]
        unit_1 = vector_1 / np.linalg.norm(vector_1)

        vector_2 = vectors[i+1]
        unit_2 = vector_2 / np.linalg.norm(vector_2)
        dot_prodct = np.dot(unit_1, unit_2)
        angle = np.arccos(dot_prodct)
        degree_angle = np.rad2deg(angle)
        angle_total += degree_angle
    if angle_total > 180:
        return True
    return False

In [24]:
def is_ring_from_positions(catheter):
    """
    This will look through a series of positions to see if it is a ring.
    Largely by checking to see if we have a gap between a series of active positions
    We should have at least 3 positions on the left and right, with at least 2 gap at the bottom
    """
    positions = [i for i in catheter.SourcePositions]
    active_on_left = 0
    inactive = 0
    active_on_right = 0
    reached_one_active = False
    passed_inactive = False
    for position in positions[1:]: # Skip the first one, locked to be low
        if position.DwellTime > 1:
            reached_one_active = True
            if not passed_inactive:
                active_on_left += 1
            else:
                active_on_right += 1
        elif reached_one_active:
            inactive += 1
            passed_inactive = True
    if active_on_right >= 3 and active_on_left >= 3 and inactive >= 2:
        return True
    else:
        return False

In [ ]:
# Check physical length of 9 cm, and Z from there

In [65]:
is_ring_by_angle(catheter)

True

In [63]:
np.save('./points.npy',np.asarray(points))

In [45]:
positions = [(i.X, i.Y, i.Z) for i in catheter.SourcePositions]

AttributeError: 'SourcePosition' object has no attribute 'X'

In [ ]:
[i for i in plan.SeedCollections]

In [ ]:
[i for i in plan.SolidApplicators]

In [ ]:
print(f'StructureID,TYPE,VOLUME')
for structure in plan.StructureSet.Structures:
    print(f'{structure.Id},{structure.DicomType},{structure.Volume:.2f}')

In [ ]:
dir(plan)

In [ ]:
[i.Id for i in plan.StructureSet.Structures]

In [38]:
from pyesapi import System

In [41]:
System.Windows.MessageBox.Show("Hello")

1